# 统计门数据分析

使用utils中的batch_read函数来读取门诊量数据，需要注意的是，其中有一个`is_filter`字段，用来决定是否使用过滤器对读取的数据进行过滤

我们提取了原始数据中的7个字段：
- 病人ID
- 访问日期
- 性别
- 诊断结果
- 病情描述
- 病历
- 身体检查结果


In [4]:
# -*- coding:utf-8 -*-
from utils import *
from Analyzer import OutpatientVolAnalyzer

topK = 30
# 过滤了复诊取药的数据
data = batch_read(cols=['PATIENT_ID', 'VISIT_DATE', 'SEX', 
                        'DIAG_DESC', 'ILLNESS_DESC', 'MED_HISTORY',
                        'BODY_EXAM'])
# 未过滤复诊取药的数据
no_filter_data = batch_read(cols=['PATIENT_ID', 'VISIT_DATE', 'SEX', 
                        'DIAG_DESC', 'ILLNESS_DESC', 'MED_HISTORY',
                        'BODY_EXAM'], is_filter = False)
data.head()

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,PATIENT_ID,VISIT_DATE,SEX,DIAG_DESC,ILLNESS_DESC,MED_HISTORY,BODY_EXAM
0,97251747,2016/1/1,女,抑郁症,服药1年，病情稳定。,无,无
1,96326999,2016/1/1,女,精神分裂症；抑郁症；双相情感障碍,发呆自笑、自言自语2月。,2月前，无诱因出现自言自语，发呆，自笑，有时手在空中比划，丈夫同事发觉异常，但生活可自理，照...,无
2,98762271,2016/1/1,男,焦虑抑郁状态,情绪低落、紧张、害怕3月。,3月来夜间做噩梦，惊醒后不能再次入睡困难，心慌，心急，心跳快，高兴不起来，自责，担心自己不能...,接触可，表情忧虑，担心，害怕，情绪低落。
3,98545813,2016/1/1,女,抑郁症；躯体化形式障碍,服药后喉咙干涩、烧灼感减轻，仍感嘴里发辣、涩，情绪好转，能干家务。,1年前无明显诱因开始出现心慌心急，心烦，憋闷感，又感到喉咙、嘴唇烧灼；头木；睡眠尚可，目前服...,无
6,98781460,2016/1/1,男,双相情感障碍；精神分裂样精神病；抑郁症,间断言行异常10余年，再发3天。,2002年出现情绪低落，高兴不起来，消极悲观，住院10余天，恢复正常。此后一切如常。2015...,精神检查：意识清，检查合作，存在被害被监控妄想，自知力不存在。


OutpatientVolAnalyzer是一个用于对门诊量数据进行分析的类，我们实现了三种方法对词按照其重要性进行排序的方法，分别是：
- TF-IDF
- Textrank
- Frequency（词出现的频率）

# 病情描述字段的分词结果

下面是对病情描述字段进行分析后的结果，可以发现，在病人描述字段中，经过分词以后得到的重要程度比较高的是（三种方法的交集，未排序）：
- 躯体
- 情绪低落
- 患者
- 气短
- 月余
- 情绪
- 心理
- 言行
- 家属
- 胸闷
- 心慌
- 紧张
- 稳定
- 不稳
- 病史
- 烦躁
- 心烦
- 病情
- 不适
- 出院
- 症状
- 心急
- 郁闷
- 服药

In [5]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['ILLNESS_DESC'])
ill_des_result = compute_term_rank(analyzer, topK)
display_term_rank_result(ill_des_result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\01\AppData\Local\Temp\jieba.cache
Loading model cost 0.670 seconds.
Prefix dict has been built succesfully.


Computing TF-IDF...
Computing TextRank...
Computing Frequency...
TF-IDF results:
情绪低落:1.1807346998455355
患者:0.5032094484489054
情绪:0.4671974496040334
病史:0.4398881234416016
月余:0.41013437528985486
家属:0.3116462970775822
服药:0.24969026808357728
躯体:0.2176144849369269
病情:0.20149034190972906
言行:0.18863606200538743
出院:0.17912257381103455
胸闷:0.1593639751934881
气短:0.14685072736642862
症状:0.14210659191544353
头痛:0.09784108807868078
心理:0.094709581786899
头晕:0.08580654152536595
不适感:0.07761512958315132
兴趣:0.0745574972149116
凭空:0.07382714636461221
家人:0.06143773950226206
情绪稳定:0.056191263542061366
记忆力:0.055796739150823005
食欲:0.05366170998207188
精神分裂症:0.0472601414413687
母亲:0.04505877978938902
住院:0.04366667664899658
疼痛:0.04188150976208623
心情:0.039380793734812174
脑电图:0.037192647183836754
TextRank results:
患者:1.0
病史:0.6117576410821597
情绪低落:0.5547234065066062
月余:0.4871188765139291
情绪:0.48391957376961864
病情:0.37616870550292003
家属:0.3555650340232196
服药:0.31832844617539674
躯体:0.25884744705709234
症状:0.21884528129258

ValueError: If using all scalar values, you must pass an index

同样的，我们还可以得到诊断结果字段中，重要的词项，从下面的结果中可以看出主要包括：


In [4]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['DIAG_DESC'])
diag_des_result = compute_term_rank(analyzer, topK)
display_term_rank_result(diag_des_result)

TF-IDF results:
状态:1.3834241246381054
障碍:1.1832742677235735
精神分裂症:1.1825156488404645
焦虑症:0.802830228655431
精神障碍:0.4204652927260638
躯体:0.37573078876974836
精神病:0.31875462845134617
器质性:0.2149421837557727
情绪:0.20511442468908234
强迫症:0.14278320237143174
形式:0.12934254572458842
症状:0.12101081590695505
疾病:0.10252542997230177
强迫性:0.09647609887403412
复发性:0.06619254663510163
性神经症:0.06391809389754821
精神:0.057948081746838284
儿童:0.055320000561825525
强迫观念:0.054138395899851637
高血压:0.04766569701148739
癫痫:0.04258852951374184
功能障碍:0.036518574116245546
幻觉:0.03634877776730727
双相:0.03145116677536647
人格障碍:0.03090822918811276
脑外伤:0.030040020503053412
糖尿病:0.022560278498756973
综合征:0.022291572660060238
行为障碍:0.020131544682183235
社交:0.018673115667299626
TextRank results:
状态:1.0
障碍:0.8791738742827919
精神分裂症:0.5265213813928461
焦虑症:0.2908628504889159
躯体:0.2905947581462906
精神障碍:0.23053283627005824
情绪:0.20020823925767336
形式:0.17124235030810775
精神病:0.16268000087501305
器质性:0.11842123780092664
疾病:0.10226078621260916
症状:0.086

In [5]:
analyzer = OutpatientVolAnalyzer(no_filter_data, focused_fields = ['MED_HISTORY'])
med_history_result = compute_term_rank(analyzer, topK)
display_term_rank_result(med_history_result)
# analysis = OutpatientVolAnalyzer(data, focused_fields = ['MED_HISTORY'])
# topK = 30
# tags1 = analysis.get_tf_idf_rank(['n', 'a'], topK)
# format_print(tags1, 'TF-IDF results:')
# tags2 = analysis.get_textrank_rank(['n', 'a'], topK)
# format_print(tags2, 'TextRank results:')
# tags3 = analysis.get_freq_rank(['n', 'a'], topK)
# format_print(tags3, 'Freqency results:')
# print('top {} intersection are:{}'.format(topK, result_intersection([tags1, tags2, tags3])))

KeyboardInterrupt: 

In [ ]:
analysis = OutpatientVolAnalyzer(no_filter_data, focused_fields = ['BODY_EXAM'])
body_exam_result = compute_term_rank(analyzer, topK)
display_term_rank_result(body_exam_result)

In [14]:
import pandas as pd
df_tfidf = pd.DataFrame([dict(ill_des_result[0])]).T
df_tfidf.columns=['tfidf']
df_txtrank = pd.DataFrame([dict(ill_des_result[1])]).T
df_txtrank.columns=['textrank']
df_freq = pd.DataFrame([dict(ill_des_result[2])]).T
df_freq.columns=['freq']

In [13]:
df_tfidf.columns=['tfidf']
df_tfidf.head()

,tfidf
情绪低落,1.180735
患者,0.503209
情绪,0.467197
病史,0.439888
月余,0.410134
